In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
model_name = "TIGER-Lab/StructLM-7B"

dataset_name = "kokujin/prompts_1"

new_model = "StructLM-7B-luis"

In [3]:
# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [4]:
# Activate 8-bit precision base model loading
use_8bit = True

# Compute dtype for 4-bit base models
bnb_8bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [5]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = f"./results/{model_name}/"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 3

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 25

In [6]:
# Maximum sequence length to use
max_seq_length = 1200

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_8bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_8bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_8bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/20345 [00:00<?, ? examples/s]

In [9]:
# Train model
trainer.train(resume_from_checkpoint = False)

  0%|          | 0/6781 [00:00<?, ?it/s]

{'loss': 1.4107, 'grad_norm': 0.2513882517814636, 'learning_rate': 2.4509803921568626e-05, 'epoch': 0.0}
{'loss': 1.339, 'grad_norm': 0.151072159409523, 'learning_rate': 4.901960784313725e-05, 'epoch': 0.01}
{'loss': 0.9962, 'grad_norm': 0.30208277702331543, 'learning_rate': 7.352941176470589e-05, 'epoch': 0.01}
{'loss': 0.5361, 'grad_norm': 0.32670092582702637, 'learning_rate': 9.80392156862745e-05, 'epoch': 0.01}
{'loss': 0.3995, 'grad_norm': 0.15247255563735962, 'learning_rate': 0.00012254901960784316, 'epoch': 0.02}
{'loss': 0.3467, 'grad_norm': 0.2853965759277344, 'learning_rate': 0.00014705882352941178, 'epoch': 0.02}
{'loss': 0.3096, 'grad_norm': 0.1253095120191574, 'learning_rate': 0.0001715686274509804, 'epoch': 0.03}
{'loss': 0.3301, 'grad_norm': 0.26811644434928894, 'learning_rate': 0.000196078431372549, 'epoch': 0.03}
{'loss': 0.2839, 'grad_norm': 0.137332946062088, 'learning_rate': 0.00019999496906143586, 'epoch': 0.03}
{'loss': 0.2993, 'grad_norm': 0.1554502546787262, 'le

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 2df00b40-75c4-405b-8b96-28fe705e3fcf)') - silently ignoring the lookup for the file config.json in TIGER-Lab/StructLM-7B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in TIGER-Lab/StructLM-7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.1693, 'grad_norm': 0.14177587628364563, 'learning_rate': 0.00019240850422419457, 'epoch': 0.15}
{'loss': 0.1908, 'grad_norm': 0.3106236457824707, 'learning_rate': 0.00019194553453747627, 'epoch': 0.15}
{'loss': 0.1878, 'grad_norm': 0.1418384313583374, 'learning_rate': 0.00019146945343587128, 'epoch': 0.16}
{'loss': 0.1644, 'grad_norm': 0.15781231224536896, 'learning_rate': 0.00019098032880844975, 'epoch': 0.16}
{'loss': 0.1539, 'grad_norm': 0.13516023755073547, 'learning_rate': 0.000190478230404286, 'epoch': 0.17}
{'loss': 0.175, 'grad_norm': 0.16053356230258942, 'learning_rate': 0.00018996322982251234, 'epoch': 0.17}
{'loss': 0.1925, 'grad_norm': 0.14316949248313904, 'learning_rate': 0.00018943540050210894, 'epoch': 0.17}
{'loss': 0.155, 'grad_norm': 0.18293477594852448, 'learning_rate': 0.00018889481771143166, 'epoch': 0.18}
{'loss': 0.1608, 'grad_norm': 0.12895315885543823, 'learning_rate': 0.00018834155853747868, 'epoch': 0.18}
{'loss': 0.1578, 'grad_norm': 0.21868377923

TrainOutput(global_step=6781, training_loss=0.1218383721045874, metrics={'train_runtime': 36007.3522, 'train_samples_per_second': 0.565, 'train_steps_per_second': 0.188, 'total_flos': 9.20533750382592e+17, 'train_loss': 0.1218383721045874, 'epoch': 0.9999016957483411})

In [10]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [11]:
tokenizer.save_pretrained(new_model + "_tokenizer")

('StructLM-7B-luis_tokenizer/tokenizer_config.json',
 'StructLM-7B-luis_tokenizer/special_tokens_map.json',
 'StructLM-7B-luis_tokenizer/tokenizer.json')